# Overview

This notebook pulls down data off the ROS 2 build farm and does some preprocessing work to make it much more easy to work with. The build farm runs a variety of tests and for each test generates a single csv file that are zipped together. This notebook takes all of those notebooks, which are really rough, and merges them together into a handful of larger CSV files that are a lot easier to work with using pandas. 

There are three types of test data that come off the build farm:

1. `overhead_node` These files evaluate a single spinning ROS node in terms of cpu / memory consumption and a few other metrics. 
2. `overhead_tests` These tests examine interop between different RMW vendors where for two different vendors one acts as a publishing node and the other acts as a subscriber node. These tests profile the performance for this network confiration
3. `two_process_perf` These tests create a publisher and subscriber that use the same RMW vendor. The nodes send messages from publisher to subscriber and the whole assembly is instrumented to collect system performance and networking performance data. 

In [4]:
import numpy as np
import pandas as pd
import matplotlib
import glob as glob
# Blessed build for evaluation is August 31st 
# https://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/387/
# https://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/387/artifact/ws/test_results/buildfarm_perf_tests/*.csv/*zip*/buildfarm_perf_tests.zip
# The next block will pull down the zip file and extract it to the correct location 

In [2]:
! wget http://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/368/artifact/ws/test_results/buildfarm_perf_tests/*.csv/*zip*/buildfarm_perf_tests.zip
! mkdir ./data/build_farm/
! mkdir ./data/build_farm/raw/
! mv buildfarm_perf_tests.zip ./data/build_farm/raw/
! unzip ./data/build_farm/raw/buildfarm_perf_tests.zip -d ./data/build_farm/raw/ 

--2021-10-06 16:27:33--  http://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/368/artifact/ws/test_results/buildfarm_perf_tests/*.csv/*zip*/buildfarm_perf_tests.zip
Resolving build.ros2.org (build.ros2.org)... 13.52.151.147
Connecting to build.ros2.org (build.ros2.org)|13.52.151.147|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/368/artifact/ws/test_results/buildfarm_perf_tests/*.csv/*zip*/buildfarm_perf_tests.zip [following]
--2021-10-06 16:27:33--  https://build.ros2.org/job/Rci__nightly-performance_ubuntu_focal_amd64/368/artifact/ws/test_results/buildfarm_perf_tests/*.csv/*zip*/buildfarm_perf_tests.zip
Connecting to build.ros2.org (build.ros2.org)|13.52.151.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘buildfarm_perf_tests.zip.1’

buildfarm_perf_test     [ <=>                ]  96.

In [19]:
# First let's try to figure out blocks of data
# I.e. what are the "sets" of files we can process.
out = glob.glob("./data/build_farm/raw/*.csv")

print("Total Files: {0}".format(len(out)))    
perf_files = [f for f in out if "performance" in f]
print("Performance Files: {0}".format(len(perf_files)))
overhead_files = [f for f in out if "overhead" in f]
print("Overhead Files: {0}".format(len(overhead_files)))
two_files = [f for f in out if "two_process" in f]
print("Two Process Files: {0}".format(len(two_files)))
sync_files = [f for f in out if "_sync" in f]
print("Sync Files: {0}".format(len(sync_files)))
async_files = [f for f in out if "async" in f]
print("Async Files: {0}".format(len(async_files)))
pub_files = [f for f in out if "_pub" in f]
print("pub Files: {0}".format(len(pub_files)))
sub_files = [f for f in out if "_sub" in f]
print("sub Files: {0}".format(len(sub_files)))
node_files = [f for f in out if "node" in f]
print("node Files: {0}".format(len(node_files)))


Total Files: 270
Performance Files: 207
Overhead Files: 63
Two Process Files: 99
Sync Files: 155
Async Files: 115
pub Files: 28
sub Files: 28
node Files: 7


In [20]:
perf_cols = ['mean virtual memory (Mb)',
             'median virtual memory (Mb)',
             'virtual memory (Mb)',
             'mean cpu_usage (%)',
             'median cpu_usage (%)',
             'cpu_usage (%)',
             'mean physical memory (Mb)',
             'median physical memory (Mb)',
             'physical memory (Mb)',
             'mean resident anonymous memory (Mb)',
             'median resident anonymous memory (Mb)',
             'resident anonymous memory (Mb)']

In [21]:
# Take all of the "overhead" files and try to merge them into a single table. 
for p in node_files:
    print(p)

df = pd.read_csv(node_files[0])
df.columns = perf_cols
for p in node_files[1:]:
    temp = pd.read_csv(p)
    temp.columns = perf_cols
    df = df.append(temp)
# parse the filenames and add that data. 
df["config"] = ["_".join(n.strip('./data/build_farm/raw/overhead_node_test_results_rmw_').strip('.csv').split('_')[1:]) for n in node_files]
df["vendor"] = [n.strip('./data/build_farm/raw/overhead_node_test_results_rmw_').split('_')[0] for n in node_files]
df = df[df.columns[::-1]]
df["file_name"] = node_files
df.to_csv("./data/build_farm/node_perf.csv")
print(len(df))
print(df["file_name"])

./data/build_farm/raw/overhead_node_test_results_rmw_cyclonedds_cpp_sync.csv
./data/build_farm/raw/overhead_node_test_results_rmw_connextdds_async.csv
./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_dynamic_cpp_async.csv
./data/build_farm/raw/overhead_node_test_results_rmw_connextdds_sync.csv
./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_cpp_sync.csv
./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_dynamic_cpp_sync.csv
./data/build_farm/raw/overhead_node_test_results_rmw_fastrtps_cpp_async.csv
7
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
0    ./data/build_farm/raw/overhead_node_test_resul...
Name: file_name, dtype: object


In [22]:
def fname_to_data(fname, head="./data/build_farm/raw/overhead_test_results_rmw_",tail="_ROS2_pub.csv"):
    """
    Munge a file name into metadata. Pull out the first and seond RMW 
    along with the "flavor" information
    """
    fname = fname.replace(head,"").replace(tail,"")
    parts = fname.split("_rmw_")
    first = parts[0].split("_")
    second = parts[1].split("_")
    # format is rmw _ <name> _ <config> _ rwm _ <name2> _ <config2>
    ret_val = {}
    ret_val["first_rmw"] = first[0]
    ret_val["second_rmw"] = second[0]
    ret_val["first_flavor"] = " ".join(first[1:])
    ret_val["second_flavor"] = " ".join(second[1:])
    return(ret_val)

fname_to_data("./data/build_farm/raw/overhead_test_results_rmw_fastrtps_cpp_async_rmw_connext_cpp_ROS2_pub.csv")

{'first_flavor': 'cpp async',
 'first_rmw': 'fastrtps',
 'second_flavor': 'cpp',
 'second_rmw': 'connext'}

In [23]:
pub_sub_cols = ['mean virtual memory (Mb)',
                'median virtual memory (Mb)',
                'virtual memory (Mb)',
                'mean cpu_usage (%)',
                'median cpu_usage (%)',
                'cpu_usage (%)',
                'mean physical memory (Mb)',
                'median physical memory (Mb)',
                'physical memory (Mb)',
                'mean resident anonymous memory (Mb)',
                'median resident anonymous memory (Mb)',
                'resident anonymous memory (Mb)',
                'mean latency_mean (ms)',
                'median latency_mean (ms)',
                'Top 5% latency (ms)',
                'max ru_maxrss',
                'mean received',
                'mean sent',
                'sum lost',
                'mean system_cpu_usage (%)',
                'mean system virtual memory (Mb)']

In [24]:
# Pull out data for the pub files and repeat for sub files. 
pub_df = pd.read_csv(pub_files[0])
print(pub_files[0])

print("DF Cols {0} vs known cols {1}".format(len(pub_df.columns),len(pub_sub_cols)))    
# squish all the files into one table
pub_df.columns = pub_sub_cols
for p in pub_files[1:]:
    temp = pd.read_csv(p)
    temp.columns = pub_sub_cols
    pub_df = pub_df.append(temp)
# parse the file names into data and add them back to table. 
flavors = [fname_to_data(flavor) for flavor in pub_files]
pub_df["from_rmw"]= [flavor["first_rmw"] for flavor in flavors]
pub_df["from_rmw_flavor"]= [flavor["first_flavor"] for flavor in flavors]
pub_df["to_rmw"]= [flavor["second_rmw"] for flavor in flavors]
pub_df["to_rmw_flavor"]= [flavor["second_flavor"] for flavor in flavors]
pub_df["file_name"] = pub_files
pub_df = pub_df[pub_df.columns[::-1]]
pub_df.to_csv("./data/build_farm/pub_perf.csv")
pub_df.head()

./data/build_farm/raw/overhead_test_results_rmw_fastrtps_cpp_async_rmw_connextdds_ROS2_pub.csv
DF Cols 21 vs known cols 21


,file_name,to_rmw_flavor,to_rmw,from_rmw_flavor,from_rmw,mean system virtual memory (Mb),mean system_cpu_usage (%),sum lost,mean sent,mean received,...,mean resident anonymous memory (Mb),physical memory (Mb),median physical memory (Mb),mean physical memory (Mb),cpu_usage (%),median cpu_usage (%),mean cpu_usage (%),virtual memory (Mb),median virtual memory (Mb),mean virtual memory (Mb)
0,./data/build_farm/raw/overhead_test_results_rm...,,connextdds,cpp async,fastrtps,1407.584516,28.782439,0.0,4.48,4.48,...,8.748677,35.7383,35.7383,34.994726,2.524640,2.36271,2.242378,162.938,162.938,157.984166
0,./data/build_farm/raw/overhead_test_results_rm...,,connextdds,cpp sync,fastrtps,1409.238387,29.260103,0.0,4.48,4.52,...,8.791519,35.7266,35.7266,35.166116,2.468170,2.37668,2.392837,182.973,182.973,177.375553
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,fastrtps,dynamic cpp sync,fastrtps,1452.596774,29.148558,0.0,4.48,4.52,...,8.768519,35.6992,35.6992,35.119942,2.304305,2.24230,2.147257,199.150,199.150,192.511908
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,fastrtps,async,connextdds,1413.216129,28.785090,0.0,4.48,4.52,...,12.379824,50.5508,50.5508,49.519297,2.381900,2.30009,2.232767,239.500,239.500,232.145761
0,./data/build_farm/raw/overhead_test_results_rm...,,connextdds,async,connextdds,1423.836452,28.761548,0.0,4.48,4.52,...,12.748707,52.1602,52.1602,50.994997,2.385885,2.30415,2.265079,240.039,240.039,232.598320


In [25]:
# Now repeat for subscribersub_perf.head()
sub_df = pd.read_csv(sub_files[0])
print(sub_files[0])

print("DF Cols {0} vs known cols {1}".format(len(sub_df.columns),len(pub_sub_cols)))    

sub_df.columns = pub_sub_cols
for p in sub_files[1:]:
    temp = pd.read_csv(p)
    temp.columns = pub_sub_cols
    sub_df = sub_df.append(temp)
    
flavors = [fname_to_data(flavor,tail="_ROS2_sub.csv") for flavor in sub_files]
sub_df["from_rmw"]= [flavor["first_rmw"] for flavor in flavors]
sub_df["from_rmw_flavor"]= [flavor["first_flavor"] for flavor in flavors]
sub_df["to_rmw"]= [flavor["second_rmw"] for flavor in flavors]
sub_df["to_rmw_flavor"]= [flavor["second_flavor"] for flavor in flavors]
sub_df["file_name"] = sub_files
sub_df = sub_df[sub_df.columns[::-1]]
sub_df.to_csv("./data/build_farm/sub_perf.csv")
sub_df.head()

./data/build_farm/raw/overhead_test_results_rmw_cyclonedds_cpp_sync_rmw_cyclonedds_cpp_ROS2_sub.csv
DF Cols 21 vs known cols 21


,file_name,to_rmw_flavor,to_rmw,from_rmw_flavor,from_rmw,mean system virtual memory (Mb),mean system_cpu_usage (%),sum lost,mean sent,mean received,...,mean resident anonymous memory (Mb),physical memory (Mb),median physical memory (Mb),mean physical memory (Mb),cpu_usage (%),median cpu_usage (%),mean cpu_usage (%),virtual memory (Mb),median virtual memory (Mb),mean virtual memory (Mb)
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,cyclonedds,cpp sync,cyclonedds,1392.994194,29.240635,0.0,0.0,4.52,...,7.224323,30.1992,30.1992,28.897290,27.15715,26.8385,25.660878,144.659,144.408,140.100583
0,./data/build_farm/raw/overhead_test_results_rm...,cpp,fastrtps,cpp sync,fastrtps,1399.312581,29.788313,0.0,0.0,4.48,...,8.699758,35.6289,35.6289,34.799011,27.30095,27.2340,26.324781,199.135,199.135,192.497392
0,./data/build_farm/raw/overhead_test_results_rm...,dynamic cpp,fastrtps,dynamic cpp sync,fastrtps,1452.437419,28.696245,0.0,0.0,4.48,...,8.809506,36.3281,36.3281,35.238005,26.59005,25.9530,24.260486,199.135,199.135,192.481452
0,./data/build_farm/raw/overhead_test_results_rm...,dynamic cpp,fastrtps,cpp sync,fastrtps,1399.284839,29.283687,0.0,0.0,4.48,...,8.753965,36.0859,36.0859,35.015841,26.49885,25.7729,24.013953,199.134,199.134,192.496518
0,./data/build_farm/raw/overhead_test_results_rm...,dynamic cpp,fastrtps,cpp async,fastrtps,1398.029032,28.728961,0.0,0.0,4.48,...,8.750752,36.0977,36.0977,35.003064,27.15800,27.0010,26.017673,179.142,179.142,173.131906


In [26]:
# now aggregate the performance results, there are two types two process and and "results"
two_process_perf = [p for p in perf_files if "two_process" in p]
result_perf_file = [p for p in perf_files if "two_process" not in p]
print("{0} two process files and {1} results files. {2} total files.".format(len(two_process_perf),len(result_perf_file),len(perf_files)))

# From: https://github.com/ahcorde/buildfarm_perf_tests/blob/master/test/test_performance.py.in#L48
perf_col_names = [
    'mean latency_mean (ms)',
    'median latency_mean (ms)',
    '95th Percentile Latency',
    'max ru_maxrss',
    'mean received',
    'mean sent',
    'sum lost',
    'mean cpu_usage (%)',
    '95th Percentile CPU',
    'median cpu_usage (%)',
    'mean data_received (Mb)',
    'median data_received (Mb)',
    '95th Percentile Data Received (Mb)']


99 two process files and 108 results files. 207 total files.


In [27]:
def fname_to_rmw_and_data(fname):
    """
    Parse and return file names of the format
    performnace_test_resuts_<optional rmw>_<rmw_name>_<rmw_flavor>_<datatype>.csv
    E.g. 
    ./data/performance_test_results_rmw_fastrtps_dynamic_cpp_async_Array32k.csv
    ./data/performance_test_results_FastRTPS_sync_Array2m.csv
    ./data/performance_test_results_CycloneDDS_sync_Array1k.csv
    """
    fname = fname.replace("./data/build_farm/raw/performance_test_two_process_results_rmw_","")
    fname = fname.replace("./data/build_farm/raw/performance_test_two_process_results_","")
    fname = fname.replace("./data/build_farm/raw/performance_test_results_","")
    
    fname = fname.replace(".csv","")
    parts = fname.split("_");
    ret_val = {}
    ret_val["type"] = parts[-1] # last entry is type, easy
    if(parts[0] == "rmw"):
        parts = parts[1:] # drop the first value if it is RMW
    ret_val["vendor"] = parts[0].lower() # both upper and lower is present
    ret_val["flavor"] = "_".join(parts[1:-1])
    return ret_val 

In [28]:
perf_df = pd.read_csv(result_perf_file[0])
print(result_perf_file[0])

print("DF Cols {0} vs known cols {1}".format(len(perf_df.columns),len(perf_col_names)))

perf_df.columns = perf_col_names

# smush main csv files together
for p in result_perf_file[1:]:
    temp = pd.read_csv(p)
    temp.columns = perf_col_names
    perf_df = perf_df.append(temp)
# parse file names 
fname_data = [fname_to_rmw_and_data(p) for p in result_perf_file]
perf_df["vendor"] = [p["vendor"] for p in fname_data]
perf_df["flavor"] = [p["flavor"] for p in fname_data]
perf_df["data_type"] = [p["type"] for p in fname_data]
perf_df["file_name"] = result_perf_file
perf_df = perf_df[perf_df.columns[::-1]]
perf_df.to_csv("./data/build_farm/perf_network_results.csv")
perf_df.head()


./data/build_farm/raw/performance_test_results_rmw_cyclonedds_cpp_sync_Array4m.csv
DF Cols 13 vs known cols 13


,file_name,data_type,flavor,vendor,95th Percentile Data Received (Mb),median data_received (Mb),mean data_received (Mb),median cpu_usage (%),95th Percentile CPU,mean cpu_usage (%),sum lost,mean sent,mean received,max ru_maxrss,95th Percentile Latency,median latency_mean (ms),mean latency_mean (ms)
0,./data/build_farm/raw/performance_test_results...,Array4m,cpp_sync,cyclonedds,4000.993488,4000.027690,4000.008750,20.980,21.230,21.051111,0.0,999.518519,999.518519,88952.0,0.824120,0.81680,0.817067
0,./data/build_farm/raw/performance_test_results...,Array4m,sync,fastrtps,4003.332180,3999.302583,4000.035998,16.500,16.925,16.470741,0.0,999.222222,999.222222,94828.0,0.646630,0.63460,0.635333
0,./data/build_farm/raw/performance_test_results...,PointCloud512k,sync,cyclonedds,500.645658,500.147693,500.238602,2.499,2.500,2.388741,0.0,999.148148,999.185185,87272.0,0.075719,0.07319,0.072183
0,./data/build_farm/raw/performance_test_results...,Array32k,async,connextdds,31.267012,31.265079,31.265150,2.248,2.498,2.210926,0.0,999.444444,999.444444,89796.0,0.058847,0.05603,0.055599
0,./data/build_farm/raw/performance_test_results...,Array2m,cpp_sync,fastrtps,2000.143338,2000.033538,2000.014709,6.244,6.744,6.354704,0.0,999.518519,999.518519,93572.0,0.250920,0.23270,0.232415


In [33]:
twop_df = pd.read_csv(two_process_perf[0])
print(two_process_perf[0])

print("DF Cols {0} vs known cols {1}".format(len(twop_df.columns),len(perf_col_names)))

twop_df.columns = perf_col_names

# smush main csv files together
for p in two_process_perf[1:]:
    temp = pd.read_csv(p)
    temp.columns = perf_col_names
    twop_df = twop_df.append(temp)
# parse file names 
fname_data = [fname_to_rmw_and_data(p) for p in two_process_perf]
twop_df["vendor"] = [p["vendor"] for p in fname_data]
twop_df["flavor"] = [p["flavor"] for p in fname_data]
twop_df["data_type"] = [p["type"] for p in fname_data]
twop_df["file_name"] = two_process_perf
twop_df = twop_df[twop_df.columns[::-1]]
twop_df.to_csv("./data/build_farm/two_process_perf_network_results.csv")
twop_df.head()


./data/build_farm/raw/performance_test_two_process_results_rmw_connextdds_sync_Array60k.csv
DF Cols 13 vs known cols 13


,file_name,data_type,flavor,vendor,95th Percentile Data Received (Mb),median data_received (Mb),mean data_received (Mb),median cpu_usage (%),95th Percentile CPU,mean cpu_usage (%),sum lost,mean sent,mean received,max ru_maxrss,95th Percentile Latency,median latency_mean (ms),mean latency_mean (ms)
0,./data/build_farm/raw/performance_test_two_pro...,Array60k,sync,connextdds,58.611541,58.608819,58.552577,1.2490,1.748,1.425519,0.0,0.0,998.518519,90256.0,0.071680,0.06920,0.068812
0,./data/build_farm/raw/performance_test_two_pro...,Array2m,sync,connextdds,5.824308,1.886525,1.883114,14.7500,19.552,14.531778,27745.0,0.0,0.444444,810852.0,42.335000,31.72000,19.463704
0,./data/build_farm/raw/performance_test_two_pro...,Array8m,async,fastrtps,150.370686,39.993253,41.770525,0.4999,2.350,0.694367,20394.0,0.0,4.555556,87684.0,29.829000,24.86000,17.406667
0,./data/build_farm/raw/performance_test_two_pro...,Array4k,dynamic_cpp_sync,fastrtps,3.924328,3.921481,3.917004,0.7493,0.999,0.835667,0.0,0.0,998.333333,93724.0,0.023204,0.02256,0.022504
0,./data/build_farm/raw/performance_test_two_pro...,Array16k,cpp_async,fastrtps,15.641148,15.640306,15.626360,0.9990,1.249,0.991170,0.0,0.0,998.666667,93548.0,0.046655,0.04491,0.044976


In [30]:
total = len(pub_files)+len(sub_files)+len(node_files)+len(two_process_perf)+len(result_perf_file)
print("processed {0} of {1}".format(total,len(glob.glob("./data/build_farm/raw/*.csv"))))

processed 270 of 270
